In [1]:
import meep as mp
import argparse
import math

In [8]:

resolution = 200
theta = 0

dpml = 1.0              # PML thickness
sz = 10                 # size of cell (without PMLs)
sz = 10 + 2*dpml
cell_size = mp.Vector3(0,0,sz)
pml_layers = [mp.PML(dpml)]

wvl_min = 0.4           # min wavelength
wvl_max = 0.8           # max wavelength
fmin = 1/wvl_max        # min frequency
fmax = 1/wvl_min        # max frequency
fcen = 0.5*(fmin+fmax)  # center frequency
df = fmax-fmin          # frequency width
nfreq = 50              # number of frequency bins

# rotation angle (in degrees) of source: CCW around Y axis, 0 degrees along +Z axis
theta_r = math.radians(theta)

# plane of incidence is xz
k = mp.Vector3(math.sin(theta_r),0,math.cos(theta_r)).scale(fmin)

# if normal incidence, force number of dimensions to be 1
if theta_r == 0:
    dimensions = 1
else:
    dimensions = 3

In [15]:
sources = [mp.Source(mp.GaussianSource(fcen,fwidth=df), component=mp.Ex, center=mp.Vector3(0,0,-0.5*sz+dpml))]

sim = mp.Simulation(cell_size=cell_size,
                    boundary_layers=pml_layers,
                    sources=sources,
                    k_point=k,
                    dimensions=dimensions,
                    resolution=resolution)

refl_fr = mp.FluxRegion(center=mp.Vector3(0,0,-0.25*sz))
refl = sim.add_flux(fcen, df, nfreq, refl_fr)

sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ex, mp.Vector3(0,0,-0.5*sz+dpml), 1e-9))

empty_flux = mp.get_fluxes(refl)
empty_data = sim.get_flux_data(refl)
sim.reset_meep()

# add a block with n=3.5 for the air-dielectric interface
geometry = [mp.Block(mp.Vector3(mp.inf,mp.inf,0.5*sz), center=mp.Vector3(0,0,0.25*sz), material=mp.Medium(index=3.5))]

sim = mp.Simulation(cell_size=cell_size,
                    geometry=geometry,
                    boundary_layers=pml_layers,
                    sources=sources,
                    k_point=k,
                    dimensions=dimensions,
                    resolution=resolution)


-----------
Initializing structure...
time for choose_chunkdivision = 0.000138044 s
Working in 3D dimensions.
Computational cell is 0.005 x 0.005 x 12 with resolution 200
time for set_epsilon = 0.0141771 s
-----------
Meep: using complex fields.
on time step 13231 (time=33.0775), 0.000302351 s/step
field decay(t = 50.0025): 0.2502050785798166 / 0.2502050785798166 = 1.0
on time step 27485 (time=68.7125), 0.00028063 s/step
field decay(t = 100.0025): 1.047412054743179e-14 / 0.2502050785798166 = 4.186214207514775e-14
run 0 finished at t = 100.0025 (40001 timesteps)


In [17]:
sim.plot3D()

TraitError: The 'scalar_data' trait of an ArraySource instance must be a 2D or 3D numpy array, but a value of array([ 9.54779412,  1.39473684,  1.        , ..., 12.25      ,
       12.25      ,  9.54779412]) <class 'numpy.ndarray'> was specified.

In [14]:
refl = sim.add_flux(fcen, df, nfreq, refl_fr)
sim.load_minus_flux_data(refl, empty_data)

sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ex, mp.Vector3(0,0,-0.5*sz+dpml), 1e-9))

refl_flux = mp.get_fluxes(refl)
freqs = mp.get_flux_freqs(refl)

for i in range(nfreq):
    print("refl:, {}, {}, {}, {}".format(k.x,1/freqs[i],math.degrees(math.asin(k.x/freqs[i])),-refl_flux[i]/empty_flux[i]))

field decay(t = 50.0025): 0.2502050784849634 / 0.2502050784849634 = 1.0
field decay(t = 100.0025): 2.8160152403534873e-13 / 0.2502050784849634 = 1.125482846873039e-12
run 0 finished at t = 100.0025 (40001 timesteps)
refl:, 0.0, 0.8, 0.0, 0.30780697376583877
refl:, 0.0, 0.784, 0.0, 0.3077724750611765
refl:, 0.0, 0.7686274509803922, 0.0, 0.30773725191827517
refl:, 0.0, 0.7538461538461539, 0.0, 0.3077012971078069
refl:, 0.0, 0.739622641509434, 0.0, 0.3076646698643559
refl:, 0.0, 0.7259259259259259, 0.0, 0.307627349476206
refl:, 0.0, 0.7127272727272727, 0.0, 0.30758929720167066
refl:, 0.0, 0.7, 0.0, 0.30755056365747957
refl:, 0.0, 0.6877192982456141, 0.0, 0.3075111443814758
refl:, 0.0, 0.6758620689655173, 0.0, 0.30747097892958153
refl:, 0.0, 0.664406779661017, 0.0, 0.3074300999308563
refl:, 0.0, 0.6533333333333333, 0.0, 0.3073885324305411
refl:, 0.0, 0.6426229508196721, 0.0, 0.3073462182598253
refl:, 0.0, 0.632258064516129, 0.0, 0.30730317509899524
refl:, 0.0, 0.6222222222222222, 0.0, 0.30